# FIFA Moneyball

In [ ]:
# Libraries
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [ ]:
#Read the dataset provided

all_players = pd.read_csv("./fifa21_male2.csv")

In [ ]:
#Check all the columns avaliable

list(all_players.columns)

In [ ]:
#Value & Release Clause Column
def format_money(column):
    values = []
    for value in all_players[column]:
        if value[-1]=='M':
            money = 1000000
            money *= float(value[1:-1])
        elif value[-1]=='K':
            money = 1000
            money *= float(value[1:-1])
        else: 
            money = 0
        values.append(money)
    return values

def format_release_clause():
    release_clause = []
    for clause in all_players['Release Clause'].fillna(''):
        if clause == '':
            money=0.0
        elif clause[-1]=='M':
            money = 1000000
            money *= float(clause[1:-1])
        elif clause[-1]=='K':
            money = 1000
            money *= float(clause[1:-1])
        else: 
            money = 0
        release_clause.append(money)
    return release_clause

all_players['Value'] =  format_money('Value')
all_players['Wage'] = format_money('Wage')
all_players['Release Clause'] = format_release_clause()

In [ ]:
(all_players).describe()

In [ ]:
#Get a quick view of the dataset and the type of information contained 

all_players.head()

In [ ]:
#Aplying some changes to the base dataset
all_players['BP'] = all_players['BP'].astype(str)
all_players.fillna(0, inplace=True)

In [ ]:
#This specific column is quite messy and difficult to work with, we will try to get the contract dates separately although we know it won't work in all cases.
# We will see later if this is enough for our analysis or if it is necessary to invest more time correcting it.

all_players[['start_year', 'end_year']] = all_players['Contract'].str.extract('(\d{4})\D+(\d{4})?')
all_players[['start_year', 'end_year']] = all_players[['start_year', 'end_year']].fillna(0)
all_players[['start_year', 'end_year']] = all_players[['start_year', 'end_year']].astype(int)


## Problem definition and analysis approach

Our assigned group is **AC Milan**, so we have decided to focus our analysis on the characteristics of this Club, who are the players in this team and how we can improve performance while saving budget.

## Filtered dataset

In [ ]:
#After checking the columns contained in the original dataset, we decided that the following fields were the most relevant fields for our analysis

players_info =all_players[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause', 'Total Stats', 'Team & Contract','start_year', 'end_year']]

In [ ]:
players_info.head()

In [ ]:
players_info.isnull()
players_info.isnull().sum()

#### Descriptions

* **BP**: Best position, some players can play in several positions, but we want to take the performance based in the best position.
* **BOV**: Best Overall Rating.
* **POT**: Potential.
* **Value**: What is the value of this player in the market.
* **Release Clause**: How much will the club be paid if this player goes to other club.
* **Total Stats**: A summary of all the different characteristics from the player.

In [ ]:
display(players_info.dtypes)

In [ ]:
players_info.describe()

In general we can see that the average age for a football player is 25 years old, and the average of the BOV and POT is 67 points and 72 points respectively with total stats around 1631.

In [ ]:
#Check how are all these characteristics related

plt.matshow(players_info.corr())
plt.xticks(range(9),players_info.corr().columns,rotation=90)
plt.yticks(range(9),players_info.corr().columns)
plt.colorbar()
plt.show()

From this grahp we get the following conclusions:
* Age vs POT: these are negative related since the older the player, the less growth potential he has.
* BOV vs Total Stats: the better stats the player has, the higher BOV he gets, making him a top player.
* Wage vs BOV: the salaray of the player is related to how good he is, so the higher the BOV the higher the wage.

## AC Milan players

Now we are going to analayze the players in the AC Milan club to try to identify what improvements can be done.

In [ ]:
#Segregate the players that belong to the club

ac_milan_players = players_info[players_info['Club'] == 'Milan']

In [ ]:
#Since the potential of the players decreases with the age, we want to see which are the older players in the team 

ac_milan_players = ac_milan_players.sort_values(by='Age', ascending= False)
ac_milan_players

We see that for some of those older players there is no wage asigned, also, their contrat seems to be expired, which makes them "non-active players". In that case, we have to exclude them.

In [ ]:
ac_milan_players = ac_milan_players.loc[(ac_milan_players.Wage > 0.00)]
ac_milan_players = ac_milan_players.loc[(ac_milan_players.end_year >= 2021)]

In [ ]:
ac_milan_players

In [ ]:
ac_milan_players['BP'].value_counts().sum()

We have 33 active players in the team, considering that we need more or less 22 players for a match, we think there might be some players that we could release. For that, first we are going to check the amount of players we have per position.

#### How does each position play?

* **GK**: A goalkeeper **(GK)** is essentially a team’s last line of defence. Goalkeeper is the most defensive position in football.
* **Defenders - Centre backs**: Centre backs **(CB)** or central defenders are centrally placed defenders just in front of the team’s goal. 
* **Defenders - Full backs**: The defenders who operate from either side of their centre backs are termed as full backs or side backs. they are further distinguished as a left back **(LB)** or a right back **(RB)**.
* **Central midfielders**: Midfielders located centrally on the pitch are called central midfielders **(CM)**.The number of central midfielders mostly depends on the team’s strategy. 
* **Central defensive midfielders** **(CDM)**: is tasked to primarily defend by staying in front of their backline. Players specialising as CDMs mostly have a high work rate and are good at tackling and interceptions.  
* **Central attacking midfielders** **(CAM)**: Are More attack-minded central midfielders who prioritise supporting the forwards in attack rather than defence. CAMs generally possess the ability to play killer final passes and can shoot well from distance.
* **Wingers**: Midfielders who operate from the flanks just ahead of the full backs are called wingers. They are called left wingers **(LW)** or right wingers **(RW)** according to their position on the pitch. While they do help their full backs defensively, a winger’s primary task is to attack and take on opposition defenders.
* **Forwards or strikers**: are the team’s most advanced players on the pitch, whose job is exclusively to score goals. A forward who plays more centrally just in front of the opposition goal is called a centre forward **(CF)**. Some teams often deploy forwards in the lane between the wingers and the centre forwards. These strikers are referred to as right forwards **(RF)** or left forwards **(LF)** according to which side they are playing from.


![Field_positions](./Field_positions.png)

#### AC Milan game strategy

Stefano Pioli (manager of the club) likes to use a 4-2-3-1 formation in which the fullbacks **-LB RB-** pick up advanced roles on the pitch. As the game progresses, their fullbacks take up offensive positions, often ending up in or near the striker position. 

Defensively, one of their fullbacks  **-LB RB-**  tucks inside with the two central defenders **-CB-** turning it into a back three. One of the midfielders **-CDM CAM-** enters the vacated space left by the fullback. When they get the ball back, they immediately attack by initiating quick passes and their fullbacks **-LB RB-** end up near the striking positions in the last phase of their offensive play.

When the team attacks, both the fullbacks **-LB RB-**  take up advanced positions in the center of the pitch. The two midfielders  **-CDM CAM-** cover the space vacated by the fullbacks, leading to a type of front six and a back four. The team also likes to exploit width by allowing their wingers **-LW RW-** to stay in wide positions.

Sometimes, the attacking midfielders **-CAM-** drop into wide positions and one of the defensive midfielders **-CDM-** comes up to that side and the three form a triangle, with quick interchange of passes and establish a 3v1 or a 3v2 dominance. It is because of this that they can put pin-point crosses to the center of the pitch and a fullback **-LB RB-** usually ends up in that center space.

In [ ]:
#We have a lot of CAM players and CB players in the team to sustain the defensive strategy of the club, however at the time we are lacking for LW and RW positions

ac_milan_players.groupby(['BP']).agg({'BOV':('mean','count'),'Age':'mean','Wage':'mean' })

In [ ]:
#Now we take a look in the players age and BOV

plt.scatter(ac_milan_players['Age'],ac_milan_players['BOV'], s = 100)
plt.show()

Majority of the players have BOV between 75 and 80, and are between ages of 22 and 30. From this perspective, we should check the POT of the younger players and the the contracts of the olders, the BOV for the olders is still between average of the team, but we know that they probably retire soon. While there isn't an exact age for footballers to retire, the largest host of professional players hang their boots between the ages of 33 and 37, with 35 being the accepted average.

In [ ]:
sns.boxplot(x='BP',y='BOV', data = ac_milan_players)
plt.show()

From the graph we can take that there are some outliers regarding BOV score in the different positions, but we already saw that we have some younger members of the team that are not yet in their full potential. So we will need to deep dive in each of the positions teams to make a better assesment.

In order to compare with the rest of the clubs positons stats, we are going to exclude the inactive players from there, and of course exclude the players from AC Milan.

In [ ]:
active_players_info = players_info.loc[(players_info.Wage > 0.00)]
active_players_info = players_info.loc[(players_info.Club != 'Milan')]

In [ ]:
general_stats = active_players_info.groupby(['BP']).agg({'BOV':'mean'})

In [ ]:
ac_milan_stats = ac_milan_players.groupby(['BP']).agg({'BOV':'mean'})

In [ ]:
stats_comparison = pd.merge(general_stats, ac_milan_stats, on='BP', how='left')
stats_comparison.fillna(0, inplace=True)
stats_comparison = stats_comparison.rename(columns = {'BOV_x' : 'General avg. BOV', 'BOV_y' : 'AC Milan avg. BOV'})

In [ ]:
stats_comparison['Comparison'] = ((((stats_comparison['AC Milan avg. BOV'] - stats_comparison['General avg. BOV'])/stats_comparison['AC Milan avg. BOV'])*100))
stats_comparison['Comparison'] = (stats_comparison['Comparison'].replace(-np.inf, -100))
stats_comparison['Comparison'] = (stats_comparison['Comparison'].astype(int)).astype(str) + '%'

In [ ]:
stats_comparison

In average it seems like the team is better than the general population of the rest of the players, negative values are coming from not avaliable positions in the club. So now lets deep dive in detail per possition to try and find a way to improve or manatain the performance and find the missing players while saving some budget.

## Current budget

In [ ]:
ac_milan_players

In [ ]:
wages_expense = (ac_milan_players['Wage'].sum())*12
wages_expense

## Analysis per position

In order to verify the specific characteristics of the players according to the position they play, we have added many more details to a new dataframe.

In [ ]:
stats_check = all_players[['ID',
 'Name',
 'Age',
 'Club',
 'BOV',
 'BP',
 'POT',
 'Team & Contract',
 'start_year', 
 'end_year',
 'Height',
 'Weight',
 'foot',
 'Loan Date End',
 'Value',
 'Wage',
 'Release Clause',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading Accuracy',
 'Short Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK Accuracy',
 'Long Passing',
 'Ball Control',
 'Movement',
 'Acceleration',
 'Sprint Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing Tackle',
 'Sliding Tackle',
 'Goalkeeping',
 'GK Diving',
 'GK Handling',
 'GK Kicking',
 'GK Positioning',
 'GK Reflexes',
 'Total Stats']]

In [ ]:
stats_check = stats_check.loc[(players_info.Wage > 0.00)]
stats_check.head()

#### (GK) Goalkeeper
Lets explore which are the top 10 goalkeepers from the population using the stats considered relevant to this position.

In [ ]:
GK_stats = stats_check[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause', 'Total Stats','Goalkeeping','GK Diving','GK Handling','GK Kicking','GK Positioning','GK Reflexes', 'start_year', 'end_year',]]
GK_stats = GK_stats.loc[(GK_stats.BP == 'GK' )]
GK_stats  = GK_stats.sort_values(by='BOV', ascending= False)
GK_stats.head(10)

In [ ]:
#How many GK the other clubs have in their teams

qty_players_GK = GK_stats.groupby(['Club']).agg({'Name':'count'})
averages = qty_players_GK['Name'].mean()
print(averages)

In [ ]:
GK_ac_milan = GK_stats.loc[(GK_stats.Club == 'Milan' )]
GK_ac_milan = GK_ac_milan.loc[(GK_stats.end_year >= 2021)]  #We apply this filter here, because for the AC Milan club it was posssible to use the field to identify active players 
GK_ac_milan = GK_ac_milan.reset_index(drop=True)
GK_ac_milan

We can see from the data that:
* We have 3 players for this position, in average the rest of the teams have 2.
* Player with low BOV and low POT, around 30 years old. Contract expires this year

Given his low stats, we will include A. Donnarumma as a posible candidate for release.

In [ ]:
players_to_release = pd.DataFrame()

# A. Donnarumma is located in the index 2

new_row = GK_ac_milan.iloc[[2],[0,1,3,4,5,6,7,8,9]]
players_to_release  = pd.concat([players_to_release, new_row], ignore_index=True)

#### (CB) Defenders - Centre backs
Lets explore which are the top 10 centre backs from the population using the stats considered relevant to this position.

In [ ]:
CB_stats = stats_check[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause', 'Total Stats','Interceptions','Defending','Agility','Balance','Strength', 'start_year', 'end_year',]]
CB_stats = CB_stats.loc[(CB_stats.BP == 'CB' )]
CB_stats  = CB_stats.sort_values(by='BOV', ascending= False)
CB_stats.head(10)

In [ ]:
#How many CB the other clubs have in their teams

qty_players_CB = CB_stats.groupby(['Club']).agg({'Name':'count'})
averages = qty_players_CB['Name'].mean()
print(averages)

In [ ]:
CB_ac_milan = CB_stats.loc[(CB_stats.Club == 'Milan' )]
CB_ac_milan = CB_ac_milan.loc[(CB_stats.end_year >= 2021)]  #We apply this filter here, because for the AC Milan club it was posssible to use the field to identify active players 
CB_ac_milan  = CB_ac_milan.reset_index(drop=True)
CB_ac_milan

We can see from the data that:
* We have 8 players for this position, in average the rest of the teams have 4.
* Couple of players with low BOV and low POT.

Since in general we have a better average than the rest of the population, for this position, we are going to check the scores of these players against the 75 quartile from the rest of the players, to determine which ones are lower that the average of the best.

In [ ]:
Q75_BOV = CB_stats['BOV'].quantile(0.75)
print(Q75_BOV)
Q75_POT = CB_stats['POT'].quantile(0.75)
print(Q75_POT)

In [ ]:
CB_players_release1 = CB_ac_milan.loc[(CB_ac_milan.BOV < Q75_BOV)] 
CB_players_release1 = CB_players_release1.reset_index(drop=True)
CB_players_release1

In [ ]:
CB_players_release2 = CB_players_release1.loc[(CB_players_release1.POT < Q75_POT)] 
CB_players_release2 = CB_players_release2.reset_index(drop=True)
CB_players_release2

In [ ]:
# G. Bellodi is located in the index 0

new_row = CB_players_release2.iloc[[0],[0,1,3,4,5,6,7,8,9]]
players_to_release  = pd.concat([players_to_release, new_row], ignore_index=True)

Aditionally, in the players to release we will include the bottom player from the Q75_BOV. His potential is above Q75_POT, however we want to be more eficient and we have too many players for this position. 

In [ ]:
# G. Bellodi is located in the index 1

new_row = CB_players_release1.iloc[[1],[0,1,3,4,5,6,7,8,9]]
players_to_release  = pd.concat([players_to_release, new_row], ignore_index=True)

#### (LB- RB) Defenders - Full backs
Lets explore which are the top 10 full backs from the population using the stats considered relevant to this position.

In [ ]:
LB_RB_stats = stats_check[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause', 'Total Stats','Aggression','Heading Accuracy','FK Accuracy','Dribbling','Crossing','foot', 'start_year', 'end_year',]]
LB_RB_stats = LB_RB_stats.loc[(LB_RB_stats.BP == 'LB' ) | (LB_RB_stats.BP == 'RB')]
LB_RB_stats = LB_RB_stats .sort_values(by='BOV', ascending= False)
LB_RB_stats.head(10)

In [ ]:
#How many LB_RB the other clubs have in their teams

qty_players_LB_RB = LB_RB_stats.groupby(['Club']).agg({'Name':'count'})
averages = qty_players_LB_RB['Name'].mean()
print(averages)

In [ ]:
LB_RB_ac_milan = LB_RB_stats.loc[(LB_RB_stats.Club == 'Milan' )]
LB_RB_ac_milan = LB_RB_ac_milan.loc[(LB_RB_stats.end_year >= 2021)]  #We apply this filter here, because for the AC Milan club it was posssible to use the field to identify active players 
LB_RB_ac_milan

In [ ]:
Q75_BOV = LB_RB_stats['BOV'].quantile(0.75)
print(Q75_BOV)
Q75_POT = LB_RB_stats['POT'].quantile(0.75)
print(Q75_POT)

In [ ]:
LB_RB_players_release = LB_RB_ac_milan.loc[(LB_RB_ac_milan.BOV < Q75_BOV)]
LB_RB_players_release

In [ ]:
LB_RB_players_release = LB_RB_players_release.loc[(LB_RB_players_release.POT < Q75_POT)] 
LB_RB_players_release

For this position, we have around the same number as the rest of the clubs, and all our players are above average of the best. So for this position we won't be doing modifications.

#### (CM LM RM) Midfielders
Lets explore which are the top 10 central midfielders from the population using the stats considered relevant to this position.

In [ ]:
M_stats = stats_check[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause', 'Total Stats','Standing Tackle','Sliding Tackle','Interceptions','Stamina', 'start_year', 'end_year',]]
M_stats = M_stats.loc[(M_stats.BP == 'CM' ) | (M_stats.BP == 'LM') | (M_stats.BP == 'RM')]
M_stats = M_stats .sort_values(by='BOV', ascending= False) 
M_stats.head(10)

In [ ]:
#How many CM the other clubs have in their teams

qty_players_M = M_stats.groupby(['Club']).agg({'Name':'count'})
averages = qty_players_M['Name'].mean()
print(averages)

In [ ]:
M_ac_milan = M_stats.loc[(M_stats.Club == 'Milan' )]
M_ac_milan = M_ac_milan.loc[(M_stats.end_year >= 2021)]  #We apply this filter here, because for the AC Milan club it was posssible to use the field to identify active players 
M_ac_milan

In [ ]:
Q75_BOV = M_stats['BOV'].quantile(0.75)
print(Q75_BOV)
Q75_POT = M_stats['POT'].quantile(0.75)
print(Q75_POT)

In [ ]:
M_players_release1 = M_ac_milan.loc[(M_ac_milan.BOV < Q75_BOV)]
M_players_release1= M_players_release1.reset_index(drop=True)
M_players_release1

In [ ]:
M_players_release2 = M_players_release1.loc[(M_players_release1.POT < Q75_POT)] 
M_players_release2

In [ ]:
# M. Brescianini is located in the index 0

new_row = M_players_release1.iloc[[0],[0,1,3,4,5,6,7,8,9]]
players_to_release  = pd.concat([players_to_release, new_row], ignore_index=True)

#### (CDM) Central defensive midfielders
Lets explore which are the top 10 central defensive midfielders from the population using the stats considered relevant to this position.

In [ ]:
CDM_stats = stats_check[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause', 'Total Stats','Standing Tackle','Sliding Tackle','Interceptions','Stamina','Short Passing','Aggression', 'start_year', 'end_year',]]
CDM_stats = CDM_stats.loc[(CDM_stats.BP == 'CDM' )]
CDM_stats = CDM_stats .sort_values(by='BOV', ascending= False)
CDM_stats.head(10)

In [ ]:
#How many CDM the other clubs have in their teams

qty_players_CDM = CDM_stats.groupby(['Club']).agg({'Name':'count'})
averages = qty_players_CDM['Name'].mean()
print(averages)

In [ ]:
CDM_ac_milan = CDM_stats.loc[(CDM_stats.Club == 'Milan' )]
CDM_ac_milan = CDM_ac_milan.loc[(CDM_stats.end_year >= 2021)]  #We apply this filter here, because for the AC Milan club it was posssible to use the field to identify active players 
CDM_ac_milan

In [ ]:
Q75_BOV = CDM_stats['BOV'].quantile(0.75)
print(Q75_BOV)
Q75_POT = CDM_stats['POT'].quantile(0.75)
print(Q75_POT)

For this position, we have around the same number as the rest of the clubs, and all our players are above average of the best. So for this position we won't be doing modifications.

#### (CAM) Central attacking midfielders 
Lets explore which are the top 10 central attacking midfielders from the population using the stats considered relevant to this position.

In [ ]:
CAM_stats = stats_check[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause','Total Stats','Finishing','Dribbling','Shot Power', 'start_year', 'end_year',]]
CAM_stats = CAM_stats.loc[(CAM_stats.BP == 'CAM' )]
CAM_stats = CAM_stats .sort_values(by='BOV', ascending= False)
CAM_stats.head(10)

In [ ]:
#How many CAM the other clubs have in their teams

qty_players_CAM = CAM_stats.groupby(['Club']).agg({'Name':'count'})
averages = qty_players_CAM['Name'].mean()
print(averages)

In [ ]:
CAM_ac_milan = CAM_stats.loc[(CAM_stats.Club == 'Milan' )]
CAM_ac_milan = CAM_ac_milan.loc[(CAM_stats.end_year >= 2021)]  #We apply this filter here, because for the AC Milan club it was posssible to use the field to identify active players 
CAM_ac_milan = CAM_ac_milan.reset_index(drop=True)
CAM_ac_milan

In [ ]:
Q75_BOV = CAM_stats['BOV'].quantile(0.75)
print(Q75_BOV)
Q75_POT = CAM_stats['POT'].quantile(0.75)
print(Q75_POT)

In [ ]:
CAM_players_release1 = CAM_ac_milan.loc[(CAM_ac_milan.BOV < Q75_BOV)]
CAM_players_release1 = CAM_players_release1.reset_index(drop=True)
CAM_players_release1

In [ ]:
CAM_players_release2 = CAM_players_release1.loc[(CAM_players_release1.POT < Q75_POT)] 
CAM_players_release2

In [ ]:
# M. D. Maldini is located in the index 0

new_row = CAM_players_release1.iloc[[0],[0,1,3,4,5,6,7,8,9]]
players_to_release = pd.concat([players_to_release, pd.DataFrame(new_row, index=[0])], ignore_index=True)

#### (LW LWB - RW RWB) Wingers 
Lets explore which are the top 10 wingers from the population using the stats considered relevant to this position.

In [ ]:
LW_RW_stats = stats_check[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause', 'Total Stats','Acceleration','Sprint Speed','Dribbling','Ball Control', 'Curve', 'Finishing','foot', 'start_year', 'end_year',]]
LW_RW_stats = LW_RW_stats.loc[(LW_RW_stats.BP == 'LW' ) | (LW_RW_stats.BP == 'LWB') | (LW_RW_stats.BP == 'RW') | (LW_RW_stats.BP == 'RWB')]
LW_RW_stats = LW_RW_stats .sort_values(by='BOV', ascending= False)
LW_RW_stats.head(10)

In [ ]:
#How many LW_RW the other clubs have in their teams

qty_players_LW_RW = LW_RW_stats.groupby(['Club']).agg({'Name':'count'})
averages = qty_players_LW_RW['Name'].mean()
print(averages)

In [ ]:
LW_RW_ac_milan = LW_RW_stats.loc[(LW_RW_stats.Club == 'Milan' )]
LW_RW_ac_milan = LW_RW_ac_milan.loc[(LW_RW_stats.end_year >= 2021)]  #We apply this filter here, because for the AC Milan club it was posssible to use the field to identify active players 
LW_RW_ac_milan

Currently only has RWB players. In order to select the best candidates that we can afford, we will separate the players in LW and RW

In [ ]:
LW_stats = LW_RW_stats.loc[(LW_RW_stats.BP == 'LW')| (LW_RW_stats.BP == 'LWB')]
LW_stats = LW_stats .sort_values(by='BOV', ascending= False)
LW_stats 

In [ ]:
Q75_BOV = LW_stats['BOV'].quantile(0.75)
print(Q75_BOV)
Q75_POT = LW_stats['POT'].quantile(0.75)
print(Q75_POT)

In [ ]:
Q80_BOV = LW_stats['BOV'].quantile(0.80)
print(Q80_BOV)
Q80_POT = LW_stats['POT'].quantile(0.80)
print(Q80_POT)

Since we are not looking for the top LW, we decided to filter the possible candidates as the players with BOV between quartiles 75 and 80, we are looking for a young player with a high potential

In [ ]:
LW_stats_filtered = LW_stats.loc[(LW_stats.BOV >= Q75_BOV ) & (LW_stats.BOV <= Q80_BOV)]
LW_stats_filtered.sort_values(by='BOV', ascending= False)
LW_stats_filtered= LW_stats_filtered.reset_index(drop=True)
LW_stats_filtered.head(30)

We decided to go with 'Luan', his best position is LW and with a POT of 82 and BOV of 74, and very good stats for the rest of the abilities, having no release clause, this young player is a top choice to join the team and develop his potential.

In [ ]:
players_to_acquire = pd.DataFrame()

# Luan is located in the index 18

new_row = LW_stats_filtered.iloc[[18],[0,1,3,4,5,6,7,8,9]]
players_to_acquire   = pd.concat([players_to_acquire, new_row], ignore_index=True)

In [ ]:
RW_stats = LW_RW_stats.loc[(LW_RW_stats.BP == 'RW')]
RW_stats = RW_stats.sort_values(by='BOV', ascending= False)
RW_stats 

In [ ]:
Q75_BOV = RW_stats['BOV'].quantile(0.75)
print(Q75_BOV)
Q75_POT = RW_stats['POT'].quantile(0.75)
print(Q75_POT)

In [ ]:
Q80_BOV = RW_stats['BOV'].quantile(0.80)
print(Q80_BOV)
Q80_POT = RW_stats['POT'].quantile(0.80)
print(Q80_POT)

In [ ]:
RW_stats_filtered  = RW_stats.loc[(RW_stats.BOV >= Q75_BOV ) & (RW_stats.BOV <= Q80_BOV)]
RW_stats_filtered.sort_values(by='BOV', ascending= False)
RW_stats_filtered = RW_stats_filtered.reset_index(drop=True)
RW_stats_filtered.head(30)

We decided to go with 'L. Singh', with a POT of 80 and BOV of 73, he hasn't any release clause and with those stats this young player has a bright future in the team.

In [ ]:
# L. Singh is located in the index 22

new_row = RW_stats_filtered.iloc[[22],[0,1,3,4,5,6,7,8,9]]
players_to_acquire   = pd.concat([players_to_acquire, new_row], ignore_index=True)

#### (CF RF LF ST) Forwards or strikers 
Lets explore which are the top 10 strikers from the population using the stats considered relevant to this position.

In [ ]:
ST_stats = stats_check[['Name', 'Age', 'Club','BOV', 'BP', 'POT','Value','Wage', 'Release Clause','Total Stats','Finishing','Aggression','Positioning','Heading Accuracy','Strength', 'start_year', 'end_year',]]
ST_stats = ST_stats.loc[(ST_stats.BP == 'CF') | (ST_stats.BP == 'RF') | (ST_stats.BP == 'LF') | (ST_stats.BP == 'ST')]
ST_stats = ST_stats .sort_values(by='BOV', ascending= False)
ST_stats.head(10)

In [ ]:
#How many CF_RF_LF_ST the other clubs have in their teams

qty_players_ST = ST_stats.groupby(['Club']).agg({'Name':'count'})
averages = qty_players_ST['Name'].mean()
print(averages)

In [ ]:
ST_ac_milan = ST_stats.loc[(ST_stats.Club == 'Milan' )]
ST_ac_milan = ST_ac_milan.loc[(ST_stats.end_year >= 2021)]  #We apply this filter here, because for the AC Milan club it was posssible to use the field to identify active players 
ST_ac_milan = ST_ac_milan.reset_index(drop=True)
ST_ac_milan

In [ ]:
Q75_BOV = ST_stats['BOV'].quantile(0.75)
print(Q75_BOV)
Q75_POT = ST_stats['POT'].quantile(0.75)
print(Q75_POT)

In [ ]:
ST_players_release1 = ST_ac_milan.loc[(ST_ac_milan.BOV < Q75_BOV)]
ST_players_release1 = ST_players_release1.reset_index(drop=True)
ST_players_release1

In [ ]:
ST_players_release2 = ST_players_release1.loc[(ST_players_release1.POT < Q75_POT)] 
ST_players_release2

Acording to the data from BOV and POT all our players are above avergage of the best, however, Z. Ibrahimović is 38 years old, hence close to retire. We will be adding him to the players to release, and since he still have very good stats we will try to negociate the exchange for one of the players to adquire.

In [ ]:
# Z. Ibrahimović is located in the index 0

new_row = ST_ac_milan.iloc[[0],[0,1,3,4,5,6,7,8,9]]
players_to_release  = pd.concat([players_to_release, new_row], ignore_index=True)

We will need to add another Stricker to the team. Lets check the options

In [ ]:
Q80_BOV = ST_stats['BOV'].quantile(0.80)
print(Q80_BOV)
Q80_POT = ST_stats['POT'].quantile(0.80)
print(Q80_POT)

In [ ]:
ST_stats_filtered = ST_stats.loc[(ST_stats.BOV >= Q75_BOV )]
ST_stats_filtered.sort_values(by='BOV', ascending= False)
ST_stats_filtered = ST_stats_filtered.reset_index(drop=True)
ST_stats_filtered.head(40)

In [ ]:
# Nilmar is located in the index 37

new_row = ST_stats_filtered.iloc[[37],[0,1,3,4,5,6,7,8,9]]
players_to_acquire   = pd.concat([players_to_acquire, new_row], ignore_index=True)

Decided to go with 'Nilmar' since is a young player with a very high potential, and very good stats.

## Loans of players

As a last step, we decided to negociate the loan of 2 players 'G. Gómez' and 'Léo Duarte'. 
For the loan the conditions will be:
10% of the 'Release Clause' as a fee for the loan of the player during 1 year
If the other club decides they want to keep the player, the 'Release Clause' will have 20% discount to by paid when the contract is sign. 

In [ ]:
players_to_loan = pd.DataFrame()

# G. Gómez is located in the index 4

new_row = CB_ac_milan.iloc[[4],[0,1,3,4,5,6,7,8,9]]
players_to_loan  = pd.concat([players_to_loan, new_row], ignore_index=True)

In [ ]:
# Léo Duarte is located in the index 3

new_row = CB_ac_milan.iloc[[3],[0,1,3,4,5,6,7,8,9]]
players_to_loan  = pd.concat([players_to_loan, new_row], ignore_index=True)

## Final results

After analyzing all the positions and the stats of the players, we conclude that these are the required movements in the team:

In [ ]:
players_to_release

In [ ]:
releases_wages_expense = players_to_release['Wage'].sum()*12
releases_wages_expense

In [ ]:
releases_clauses_received =  players_to_release['Release Clause'].sum()
releases_clauses_received 

In [ ]:
players_to_acquire

In [ ]:
adquires_wages_expense = players_to_acquire['Wage'].sum()*12
adquires_wages_expense

In [ ]:
releases_clauses_paid =  players_to_acquire['Release Clause'].sum()
releases_clauses_paid

In [ ]:
players_to_loan 

In [ ]:
loan_wages_expense = players_to_loan['Wage'].sum()*12
loan_wages_expense

In [ ]:
releases_clauses_loan =  (players_to_loan['Release Clause'].sum())*0.10
releases_clauses_loan 

The budget impact is as follows:

In [ ]:
#Cash Flow from the changes

cash_flow = releases_wages_expense + releases_clauses_received - adquires_wages_expense - releases_clauses_paid + loan_wages_expense + releases_clauses_loan 
cash_flow

## New team

In [ ]:
ac_milan_players_final = ac_milan_players.sort_values(by='BOV', ascending= False).reset_index(drop=True)
ac_milan_players_final = ac_milan_players_final.drop(columns=['Team & Contract', 'start_year','end_year','Club'])
ac_milan_players_final = ac_milan_players_final.drop([27,32,28,29,22,2,23,24])
ac_milan_players_final = pd.concat([ac_milan_players_final , players_to_acquire], ignore_index=True)
ac_milan_players_final

In [ ]:
new_ac_milan_stats = ac_milan_players_final.groupby(['BP']).agg({'BOV':'mean'})

In [ ]:
final_stats_comparison = pd.merge(new_ac_milan_stats, ac_milan_stats,  on='BP', how='left')
final_stats_comparison.fillna(0, inplace=True)
final_stats_comparison = final_stats_comparison.rename(columns = {'BOV_x' : 'New AC Milan avg. BOV', 'BOV_y' : 'Old AC Milan avg. BOV'})

In [ ]:
final_stats_comparison['Comparison'] = ((((final_stats_comparison['New AC Milan avg. BOV'] - final_stats_comparison['Old AC Milan avg. BOV'])/final_stats_comparison['Old AC Milan avg. BOV'])*100))
final_stats_comparison['Comparison'] = (final_stats_comparison['Comparison'].replace(np.inf, 100))
final_stats_comparison['Comparison'] = (final_stats_comparison['Comparison'].astype(int)).astype(str) + '%'

In [ ]:
final_stats_comparison

In [ ]:
average_BOV = final_stats_comparison['Old AC Milan avg. BOV'].mean()
print(average_BOV)

In [ ]:
average_BOV_new = final_stats_comparison['New AC Milan avg. BOV'].mean()
print(average_BOV_new)

In [ ]:
#Previous expense
wages_expense

In [ ]:
new_wages_expense = (ac_milan_players_final['Wage'].sum())*12
new_wages_expense

In [ ]:
Difference = wages_expense - new_wages_expense
Difference